# JP Morgan Quantitative Research Task 2

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV
import datetime
from dateutil.relativedelta import relativedelta
from pandas import Timestamp

In [2]:
# Reading data and converting datatype
df = pd.read_csv("Nat_Gas.csv")
display(df.shape)
df.index = pd.to_datetime(df["Dates"])
df = df.drop(columns=['Dates'])
display(df.head())
df_exist_dict = df.to_dict()["Prices"]
# display(df_exist_dict)

(48, 2)

,Prices
Dates,
2020-10-31,10.1
2020-11-30,10.3
2020-12-31,11.0
2021-01-31,10.9
2021-02-28,10.9


In [3]:
df_new = pd.read_csv("Nat_Gas.csv")[36:] # Last 12 months of data
display(df_new)
print(df_new.shape)
df_new.index = [pd.to_datetime(x) + relativedelta(years=1) for x in df_new["Dates"]] # Extrapolate for 1 year into the future

df_new = df_new.drop(columns=['Dates', 'Prices'])
df_new.index.name = "Dates"

,Dates,Prices
36,10/31/23,11.8
37,11/30/23,12.2
38,12/31/23,12.8
39,1/31/24,12.6
40,2/29/24,12.4
41,3/31/24,12.7
42,4/30/24,12.1
43,5/31/24,11.4
44,6/30/24,11.5
45,7/31/24,11.6


(12, 2)


In [4]:
enc = OneHotEncoder()
X_year_month_onehot = np.hstack(
    [
        df.index.year.values.reshape(-1, 1),
        enc.fit_transform(df.index.month.values.reshape(-1, 1)).toarray(),
    ]
)
y = df.values

In [5]:
X_new = (df_new.index.astype("int64").values.reshape(-1, 1) // 10 ** 9)

X_new_year_month_onehot = np.hstack(
    [
        df_new.index.year.values.reshape(-1, 1),
        enc.transform(df_new.index.month.values.reshape(-1, 1)).toarray(),
    ]
)
poly_transformer = PolynomialFeatures(
    interaction_only=True, include_bias=False
)

X_year_month_onehot_poly = poly_transformer.fit_transform(X_year_month_onehot)
X_new_year_month_onehot_poly = poly_transformer.transform(X_new_year_month_onehot)

lr_regressor_nolag = RidgeCV()
lr_regressor_nolag.fit(X_year_month_onehot_poly, y)
# display(X_year_month_onehot_poly.shape)
df_new["Prices"] = lr_regressor_nolag.predict(X_new_year_month_onehot_poly)
display(df_new)

,Prices
Dates,
2024-10-31,12.101660
2024-11-30,12.677377
2024-12-31,13.052808
2025-01-31,13.127239
2025-02-28,13.052062
2025-03-31,13.127252
2025-04-30,12.526510
2025-05-31,12.135981
2025-06-30,12.050865


In [6]:
def pricepredict(datelist):
    """
    Predict prices for given list of dates datelist, based on their month and year
    
    Parameters:
    -----------
    datelist : array-like
        List of dates in the form %Y-%m-%d

    Returns:
    --------
    Table of prices and their corresponding values
    """
    list = pd.DatetimeIndex([pd.to_datetime(x) for x in datelist])
    table = pd.DataFrame([], index = list, columns=['Prices'])
    table.index.name = "Dates"
    val = np.hstack(
        [
            table.index.year.values.reshape(-1, 1),
            enc.transform(table.index.month.values.reshape(-1, 1)).toarray(),
        ]
    )
    table["Prices"] = lr_regressor_nolag.predict(poly_transformer.transform(val))
    return table

In [7]:
v = pricepredict(['2024-10-31', "2025-10-31"])
u = v.to_dict()["Prices"]
# w = list(v.index)
display(v)
display(v["Prices"][0])
display(u)
# print(w)
# print(min(w))
list = [pd.to_datetime(x) for x in ['2024-10-31', "2025-10-31"]]
print(list)

,Prices
Dates,
2024-10-31,12.101660
2025-10-31,12.642309


12.101659571031632

{Timestamp('2024-10-31 00:00:00'): 12.101659571031632,
 Timestamp('2025-10-31 00:00:00'): 12.642308925048155}

[Timestamp('2024-10-31 00:00:00'), Timestamp('2025-10-31 00:00:00')]


In [8]:
[pd.to_datetime(Timestamp('2024-10-31 00:00:00')), Timestamp('2024-10-31 00:00:00')]

[Timestamp('2024-10-31 00:00:00'), Timestamp('2024-10-31 00:00:00')]

In [9]:
# Define the two dates
date1 = datetime.date(2022, 1, 1)
date2 = datetime.date(2023, 6, 1)

# Calculate the number of months between the two dates
months_between = 12 * relativedelta(date2, date1).years + relativedelta(date2, date1).months

print(months_between)

17


In [10]:
# https://stackoverflow.com/questions/37297343/how-to-ensure-keys-or-values-are-of-a-specified-data-type-in-a-dict
class mydict(dict):
    def __setitem__(self, key, val):
        if not isinstance(key, datetime):
            raise ValueError("key must be datetime")
        if not isinstance(val, float): 
            raise ValueError("value must be float") 
        dict.__setitem__(self, key, val)

In [11]:
a = mydict({pd.to_datetime('2024-10-31'):3})
a

{Timestamp('2024-10-31 00:00:00'): 3}

In [12]:
b = mydict({Timestamp('2024-10-31 00:00:00'):3})
dict = {key: None for key in b.keys()}
print(dict)

{Timestamp('2024-10-31 00:00:00'): None}


## Prototype Pricing Model

In [13]:
def contractprice(injewithdates, pricelist, gasrate, maxvol, storagecost):
    """
    Given existing price data and input parameters, produce contract price. 
    Assumes that injection and withdrawal involve buying and selling of fuel respectively.
    
    Parameters:
    -----------
        
    injewithdates: dictionary
        Dictionary of injection and withdrawal dates (in the form %Y-%m-%d) and amounts.
        Injections are positive, withdrawals are negative.
        
    pricelist: dataframe 
        Prices at which the commodity can be purchased/sold on those dates.

    gasrate: float 
        Rate at which the gas can be injected/withdrawn.
        
    maxvol: float 
        Maximum volume that can be stored.
        
    storagecost: float
        Fixed fees per month for storing.

    Returns:
    --------
    Contract price. 
    """ 

    amount = 0
    dict1 = {pd.to_datetime(key): value for key, value in injewithdates.items()}
    dict2 = pricelist.to_dict()["Prices"]
    sumcosts = {key: None for key in dict1.keys()}
    for k, v in dict1.items(): 
        if (v < 0) and (amount >= abs(v)): 
            amount -= v
        elif ((amount + v) <= maxvol): 
            amount += v
        else: 
            raise Exception("Invalid storage amount")
        
        if k in df_exist_dict: 
            sumcosts[k] = v * dict2[k]
        else: 
            sumcosts[k] = v * pricepredict([k])["Prices"][0]

    display(sumcosts)
    finalexchanges = sum(sumcosts.values())
    date1 = min(dict1.keys())
    date2 = max(dict1.keys())
    nummonths = 12 * relativedelta(date2, date1).years + relativedelta(date2, date1).months
    
    totalgasprice = gasrate * sum(abs(number) for number in dict1.values())
    total_storagecost = storagecost * nummonths
    
    final_price = finalexchanges - totalgasprice - total_storagecost
    return final_price

In [14]:
injewithdates = {"2022-10-31":30,"2023-06-30":-40,"2023-10-31":10}
contractprice(injewithdates, df, 0.01, 50, 0.1)

{Timestamp('2022-10-31 00:00:00'): 330.0,
 Timestamp('2023-06-30 00:00:00'): -436.0,
 Timestamp('2023-10-31 00:00:00'): 118.0}

10.0